In [1]:
import os

import numpy as np
import pandas as pd
import numpy.testing as npt
import matplotlib.pyplot as plt
from scipy.stats import spearmanr, pearsonr

import sys
sys.path.append('..')

import modules.stats as st
import modules.plotting as pl
import modules.pandas_funcs as pf

In [2]:
results_dir = os.path.join('..', 'data', 'results')
match_dir = os.path.join('..', 'data', 'matching')


df_k_raw = pd.read_csv(os.path.join(results_dir, 'kinect_gait_metrics.csv'),
                       index_col=0)

df_z_raw = pd.read_csv(os.path.join(results_dir, 'zeno_gait_metrics.csv'),
                       index_col=0)

df_match = pd.read_csv(os.path.join(match_dir, 'match_kinect_zeno.csv'))

# Drop rows where file has no match
df_match = df_match.dropna(axis=0)


df_match_zeno = pd.merge(df_match, df_z_raw, left_on='Zeno', right_index=True)

df_total = pd.merge(df_match_zeno, df_k_raw, left_on='Kinect',
                    right_index=True, suffixes=('_z', '_k'))


# Take columns from total DataFrame to get Kinect and Zeno data
df_k = df_total.filter(like='_k')
df_z = df_total.filter(like='_z')

# Remove suffixes from column names
df_k = df_k.rename(columns=lambda x: str(x)[:-2])
df_z = df_z.rename(columns=lambda x: str(x)[:-2])


In [3]:
df_error = abs((df_k - df_z) / df_z) * 100

In [4]:
# abs_step_length_L = np.sqrt(df_z.step_length_L ** 2 + df_z.stride_width_L ** 2)
# abs(abs_step_length_L - df_z.absolute_step_length_L).max()

In [5]:
# abs_step_length_L = np.sqrt(df_k.step_length_L ** 2 + df_k.stride_width_L ** 2)
# abs(abs_step_length_L - df_k.absolute_step_length_L).idxmax()

In [6]:
high_error_index = df_error.median(axis=1).idxmax()

In [7]:
df_match.loc[high_error_index]

Zeno      B_2014-12-16_P007_Post_tc_A
Kinect       2014-12-16_P007_Post_007
Name: 28, dtype: object

In [8]:
df_compare = pd.concat([df_k.loc[high_error_index], df_z.loc[high_error_index]], axis=1)
df_compare.columns = ['Kinect', 'Zeno']
df_compare

,Kinect,Zeno
absolute_step_length_L,40.997630,49.494
absolute_step_length_R,45.467097,51.814
step_length_L,40.117590,48.764
step_length_R,44.356035,51.050
stride_length_L,78.105327,98.647
stride_length_R,91.311881,99.098
stride_time_L,1.091667,1.475
stride_time_R,1.166667,1.505
stride_velocity_L,70.023033,67.275
stride_velocity_R,78.732719,66.694
